## Installing necessary libraries

In [1]:
pip install responses

     |████████████████████████████████| 143kB 28.3MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
pip install today

  Created wheel for today: filename=today-0.2-cp37-none-any.whl size=2302 sha256=f3519d6fa7db033f29ff97690e34c6c20b8ef431c8d9e9015151a925288aa8fe
  Stored in directory: /root/.cache/pip/wheels/24/22/4d/f4edebd0c412f9f72cb89683321079f660e5c89027550c00d8
Successfully built today


#Importing libraries

In [3]:

import csv
from datetime import datetime
from bs4 import BeautifulSoup as bs
import requests
import responses

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [4]:
def get_url (position, location):

    template = "https://in.indeed.com/jobs?q={}&l={}"
    url = template.format(position, location)
    return url

In [5]:
url = get_url('data analyst','kerala')

## Extract Raw html

In [6]:
response = requests.get(url)

In [7]:
response

<Response [200]>

In [8]:
response.reason

''

#### Response 200 shows that our request is successful

In [9]:
soup = bs(response.text,'html.parser')

In [10]:
cards = soup.find_all('div','jobsearch-SerpJobCard')

In [11]:
len(cards)

15

## Protyping the model with a single record

In [12]:
card = cards[0]

In [13]:
atag = card.h2.a

In [14]:
job_title = atag.get('title')

In [15]:
job_title

'Data Analyst'

In [16]:
atag.get("href")

'/rc/clk?jk=4901a914a5935cd9&fccid=342e2af6a6c349ae&vjs=3'

In [17]:
job_url ='https://in.indeed.com' + atag.get("href")

In [18]:
job_url

'https://in.indeed.com/rc/clk?jk=4901a914a5935cd9&fccid=342e2af6a6c349ae&vjs=3'

In [19]:
card.find('span','company')

<span class="company">
AlphaExpression Technologies</span>

In [20]:
card.find('span','company').text

'\nAlphaExpression Technologies'

In [21]:
## remove space using strip ()method
company = card.find('span','company').text.strip()

In [22]:
company

'AlphaExpression Technologies'

In [23]:
card.find('div','recJobLoc')

<div class="recJobLoc" data-rc-loc="Thiruvananthapuram, Kerala" id="recJobLoc_4901a914a5935cd9" style="display: none"></div>

In [24]:
job_location = card.find('div','recJobLoc').get('data-rc-loc')

In [25]:
card.find('div','summary')

<div class="summary">
<ul style="list-style-type:circle;margin-top: 0px;margin-bottom: 0px;padding-left:20px;">
<li style="margin-bottom:0px;">Study and convert functional requirements into technical tasks.</li>
<li style="margin-bottom:0px;">Analyse, calculate and provide estimates.</li>
<li style="margin-bottom:0px;">Ensure the product meets the requirements.</li>
<li>Demonstrate ownership and initiative-taking.</li>
</ul></div>

In [26]:
job_summary1=card.find('div','summary').text.strip()

In [27]:
job_summary1

'Study and convert functional requirements into technical tasks.\nAnalyse, calculate and provide estimates.\nEnsure the product meets the requirements.\nDemonstrate ownership and initiative-taking.'

In [28]:
job_summary= job_summary1.replace("\n", " ")

In [29]:
job_summary

'Study and convert functional requirements into technical tasks. Analyse, calculate and provide estimates. Ensure the product meets the requirements. Demonstrate ownership and initiative-taking.'

In [30]:
card.find('span','date')

<span class="date date-a11y">15 days ago</span>

In [31]:
post_date = card.find('span','date').text

In [32]:
post_date

'15 days ago'

In [33]:
from datetime import date
today = date.today()
current_date= today.strftime("%B %d, %Y")
print(current_date)

June 16, 2021


In [34]:
## Some job post  doesnot have the salary range
try:
    job_salary = card.find('span','salaryText').text.strip()
except AttributeError:
    job_salary = ''

## Generalize the model with a function

In [35]:
def get_record(card):
    atag = card.h2.a
    job_title = atag.get('title')
    job_url ='https://in.indeed.com' + atag.get("href")
    company = card.find('span','company').text.strip()
    job_location = card.find('div','recJobLoc').get('data-rc-loc')
    job_summary1=card.find('div','summary').text.strip()
    job_summary = job_summary1.replace("\n", " ")
    post_date = card.find('span','date').text
    current_date= today.strftime("%B %d, %Y")
    try:
        job_salary = card.find('span','salaryText').text.strip()
    except AttributeError:
        job_salary = ''
        
    record = (job_title,company,job_location,post_date, current_date,job_summary,job_salary,job_url)
    
    return record

In [36]:
records = []

for card in cards :
    record = get_record(card)
    records.append(record)

In [37]:
records[0]

('Data Analyst',
 'AlphaExpression Technologies',
 'Thiruvananthapuram, Kerala',
 '15 days ago',
 'June 16, 2021',
 'Study and convert functional requirements into technical tasks. Analyse, calculate and provide estimates. Ensure the product meets the requirements. Demonstrate ownership and initiative-taking.',
 '',
 'https://in.indeed.com/rc/clk?jk=4901a914a5935cd9&fccid=342e2af6a6c349ae&vjs=3')

## Getting the next page

In [38]:
while True:
    try:
        url ='https://in.indeed.com' + soup.find('a',{'aria-label':'Next'}).get('href')
    except AttributeError:
        break
    response = requests.get(url)
    soup = bs(response.text,'html.parser')
    cards = soup.find_all('div','jobsearch-SerpJobCard')
    
    for card in cards:
        record = get_record(card)
        records.append(record)

In [39]:
len(records)

57

## Combining the entire scrapping process together

In [40]:
import csv
from datetime import datetime
import requests
from bs4 import BeautifulSoup as bs

def get_url(position, location):
    template = "https://in.indeed.com/jobs?q={}&l={}"
    url = template.format(position, location)
    return url
def get_record(card):
    atag = card.h2.a
    job_title = atag.get('title')
    job_url ='https://in.indeed.com' + atag.get("href")
    company = card.find('span','company').text.strip()
    job_location = card.find('div','recJobLoc').get('data-rc-loc')
    job_summary1=card.find('div','summary').text.strip()
    job_summary = job_summary1.replace("\n", " ")
    post_date = card.find('span','date').text
    current_date= today.strftime("%B %d, %Y")
    try:
        job_salary = card.find('span','salaryText').text.strip()
    except AttributeError:
        job_salary = ''
        
    record = (job_title,company,job_location,post_date, current_date,job_summary,job_salary,job_url)
    
    return record

def main(position,location):
    records =[]
    url = get_url(position,location)
    
    
    #extract the job data
    
    while True:
            response = requests.get(url)
            soup = bs(response.text,'html.parser')
            cards = soup.find_all('div','jobsearch-SerpJobCard')
          
    
    
            for card in cards:
                record = get_record(card)
                records.append(record)
        
            try:  
                url ='https://in.indeed.com' + soup.find('a',{'aria-label':'Next'}).get('href')
            except AttributeError:
                break
    # Save the the job data
    with open('job_results_1.csv','w',newline = '',encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Job Title','Company','Loaction','PostDate','CurrentDate','Summary','Salary','Joburl'])
        writer.writerows(records)

### Run the main program

In [41]:
main('software developer', 'Kerala')